In [106]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [107]:
!pip install bottleneck

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [108]:
import bottleneck as bn

In [109]:
# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
import torch
def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [110]:
from torch_geometric.nn import GATConv, SAGEConv, GraphSAGE
from torch_geometric.nn.aggr import LSTMAggregation, SortAggregation, MultiAggregation, Aggregation
from torch.nn import Module, Linear, Embedding, LSTM, MSELoss
from torch.nn import functional as F
import torch 
from torch import nn, Tensor
from typing import Optional
from torch_geometric.utils import scatter, to_dense_batch

In [111]:
def ndcg_metrics(X_pred, heldout_batch, k=100):
    '''
    normalized discounted cumulative gain@k for binary relevance
    ASSUMPTIONS: all the 0's in heldout_data indicate 0 relevance
    '''
    batch_users = X_pred.shape[0]
    idx_topk_part = bn.argpartition(-X_pred, k, axis=1)
    topk_part = X_pred[np.arange(batch_users)[:, np.newaxis],
                       idx_topk_part[:, :k]]
    idx_part = np.argsort(-topk_part, axis=1)
    # X_pred[np.arange(batch_users)[:, np.newaxis], idx_topk] is the sorted
    # topk predicted score
    idx_topk = idx_topk_part[np.arange(batch_users)[:, np.newaxis], idx_part]
    # build the discount template
    tp = 1. / np.log2(np.arange(2, k + 2))

    DCG = (heldout_batch[np.arange(batch_users)[:, np.newaxis],
                         idx_topk].toarray() * tp).sum(axis=1)
    IDCG = np.array([(tp[:min(n, k)]).sum()
                     for n in heldout_batch.getnnz(axis=1)])
    return DCG / IDCG


def recall_metrics(X_pred, heldout_batch, k=100):
    batch_users = X_pred.shape[0]

    idx = bn.argpartition(-X_pred, k, axis=1)
    X_pred_binary = np.zeros_like(X_pred, dtype=bool)
    X_pred_binary[np.arange(batch_users)[:, np.newaxis], idx[:, :k]] = True

    X_true_binary = (heldout_batch > 0).toarray()
    tmp = (np.logical_and(X_true_binary, X_pred_binary).sum(axis=1)).astype(
        np.float32)
    recall = tmp / np.minimum(k, X_true_binary.sum(axis=1))
    return recall

In [112]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import StandardScaler
data = pd.read_csv("/content/drive/MyDrive/ml-1m/ratings.dat", sep = "::", names = ["user", "item", "rating", "timestamp"])
data = data.sort_values(by = ["timestamp"])

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [113]:
data

,user,item,rating,timestamp
1000138,6040,858,4,956703932
1000153,6040,2384,4,956703954
999873,6040,593,5,956703954
1000007,6040,1961,4,956703977
1000192,6040,2019,5,956703977
...,...,...,...,...
825793,4958,2399,1,1046454338
825438,4958,1407,5,1046454443
825724,4958,3264,4,1046454548
825731,4958,2634,3,1046454548


In [114]:
groupby_users = data.groupby('user', as_index = False).agg(lambda x : len([elem for elem in x]))[["user", "item"]]
groupby_users.groupby("item", as_index = False).agg(lambda x : len([elem for elem in x])).sort_values(by = ["user"])

,item,user
742,2314,1
580,678,1
578,674,1
577,671,1
576,670,1
...,...,...
2,22,77
4,24,81
3,23,81
0,20,86


In [115]:
split = 0.9

In [116]:
user_features = pd.read_csv("/content/drive/MyDrive/ml-1m/users.dat", sep = "::", names = ["user", "gender", "age", "occupation", "zip_code"])

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [117]:
user_features

,user,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


In [118]:
user_features = user_features.drop(["zip_code"], axis = 1)

In [119]:
user_features

,user,gender,age,occupation
0,1,F,1,10
1,2,M,56,16
2,3,M,25,15
3,4,M,45,7
4,5,M,25,20
...,...,...,...,...
6035,6036,F,25,15
6036,6037,F,45,1
6037,6038,F,56,1
6038,6039,F,45,0


In [120]:
item_features = pd.read_csv("/content/drive/MyDrive/ml-1m/movies.dat", sep = "::", names = ["item", "movie_title", "genre"],
                            encoding='latin-1')
item_to_new_id = dict(np.hstack((item_features["item"].unique().reshape(-1, 1), 
                                 np.linspace(1, item_features["item"].unique().shape[0], item_features["item"].unique().shape[0]).astype(int).reshape(-1, 1))))
item_features["item"] = item_features["item"].apply(lambda x : item_to_new_id[x])

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [121]:
data["item"] = data["item"].apply(lambda x : item_to_new_id[x])

In [122]:
item_features

,item,movie_title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3879,Meet the Parents (2000),Comedy
3879,3880,Requiem for a Dream (2000),Drama
3880,3881,Tigerland (2000),Drama
3881,3882,Two Family House (2000),Drama


In [123]:
#item_features = item_features.drop(["movie_title", "release_date", "video_release_date", "IMDb_URL"], axis = 1)

In [124]:
user_category_columns = ["occupation"]
user_numeric_columns = ["age", "gender"]

In [125]:
item_category_columns = ["unknown", "Action", "Adventure", "Animation", 
                         "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy",
                         "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi",
                         "Thriller", "War", "Western"]
for column in item_category_columns[1:]:
  item_features[column] = item_features["genre"].apply(lambda x : (1 if column in x.split("|") else 0))
item_features["unknown"] = np.zeros((len(item_features),))
item_features["unknown"][item_features[item_category_columns[1:]].sum(axis = 1) == 0] = 1.
item_features = item_features.drop(["genre"], axis = 1)
item_numeric_columns = []

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [126]:
n_category = [len(user_category_columns), len(item_category_columns)]
n_numeric = [len(user_numeric_columns), len(item_numeric_columns)]

In [127]:
max_len_category = len(user_category_columns) + len(item_category_columns)

In [128]:
max_len_category

20

In [129]:
max_size_embedding_category = 0 

In [130]:
def preprocess_profiles(recent_profiles, category_columns, numeric_columns, max_size_embedding_category, start_index = 3, max_len_category = None):
    new_max_size_category = max_size_embedding_category
    profiles = recent_profiles.copy()
    profiles["gender"] = profiles["gender"].apply(lambda x : (1. if x == 'M' else 0.))
    scaler = StandardScaler() 
    profiles[numeric_columns] = scaler.fit_transform(profiles[numeric_columns])
    columns_to_numeric_id = dict([(column, {}) for column in category_columns])
    for i, column in enumerate(category_columns):
        n_distinct_val = profiles[column].nunique()
        columns_to_numeric_id[column] = dict(np.hstack((profiles[column].unique().reshape(-1, 1), 
                                                        (np.linspace(0, n_distinct_val - 1, n_distinct_val) * max_len_category + i + start_index).reshape(-1, 1).astype(int))))
        new_max_size_category = max(new_max_size_category, int((n_distinct_val - 1) * max_len_category + i + start_index))
        columns_to_numeric_id[column][0] = 0
        profiles[column] = profiles[column].apply(lambda x : columns_to_numeric_id[column][x])
    return profiles, new_max_size_category

In [131]:
def preprocess_profiles_2(recent_profiles, category_columns, numeric_columns, max_size_embedding_category, start_index = 3, max_len_category = None):
    new_max_size_category = max_size_embedding_category
    profiles = recent_profiles.copy()
    scaler = StandardScaler() 
    if len(numeric_columns) > 0:
      profiles[numeric_columns] = scaler.fit_transform(profiles[numeric_columns])
    columns_to_numeric_id = dict([(column, {}) for column in category_columns])
    for i, column in enumerate(category_columns):
        n_distinct_val = profiles[column].nunique()
        columns_to_numeric_id[column] = dict(np.hstack((profiles[column].unique().reshape(-1, 1), 
                                                        (np.linspace(0, n_distinct_val - 1, n_distinct_val) * max_len_category + i + start_index).reshape(-1, 1).astype(int))))
        new_max_size_category = max(new_max_size_category, int((n_distinct_val - 1) * max_len_category + i + start_index))
        columns_to_numeric_id[column][0] = 0
        profiles[column] = profiles[column].apply(lambda x : columns_to_numeric_id[column][x])
    return profiles, new_max_size_category

In [132]:
users_profiles, max_size_embedding_category =  preprocess_profiles(user_features, user_category_columns, user_numeric_columns, max_size_embedding_category, 3, max_len_category)   
items_profiles, max_size_embedding_category =  preprocess_profiles_2(item_features, item_category_columns, item_numeric_columns, max_size_embedding_category, 3 + len(user_category_columns), max_len_category)

In [133]:
users_profiles["gender"].nunique()

2

In [134]:
users_profiles

,user,gender,age,occupation
0,1,-1.591927,-2.298525,3
1,2,0.628170,1.966729,23
2,3,0.628170,-0.437323,43
3,4,0.628170,1.113678,63
4,5,0.628170,-0.437323,83
...,...,...,...,...
6035,6036,-1.591927,-0.437323,43
6036,6037,-1.591927,1.113678,123
6037,6038,-1.591927,1.966729,123
6038,6039,-1.591927,1.113678,0


In [135]:
max_len_id = 2

In [136]:
data["user"] = data["user"].apply(lambda x : x * max_len_id)
data["item"] = data["item"].apply(lambda x : x * max_len_id + 1)

In [137]:
users_profiles['user'] = users_profiles["user"].apply(lambda x : x * max_len_id)
items_profiles['item'] = items_profiles["item"].apply(lambda x : x * max_len_id + 1)

In [138]:
items_profiles

,item,movie_title,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,unknown
0,3,Toy Story (1995),0,0,7,8,9,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5,Jumanji (1995),0,26,0,8,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7,Grumpier Old Men (1995),0,0,0,0,9,0,0,0,...,0,0,0,0,38,0,0,0,0,0
3,9,Waiting to Exhale (1995),0,0,0,0,9,0,0,32,...,0,0,0,0,0,0,0,0,0,0
4,11,Father of the Bride Part II (1995),0,0,0,0,9,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,7759,Meet the Parents (2000),0,0,0,0,9,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3879,7761,Requiem for a Dream (2000),0,0,0,0,0,0,0,32,...,0,0,0,0,0,0,0,0,0,0
3880,7763,Tigerland (2000),0,0,0,0,0,0,0,32,...,0,0,0,0,0,0,0,0,0,0
3881,7765,Two Family House (2000),0,0,0,0,0,0,0,32,...,0,0,0,0,0,0,0,0,0,0


In [139]:
profiles_numeric = []
profiles_category = []
profiles_numeric.append({users_profiles.values[i, 0] : users_profiles[user_numeric_columns].values[i] for i in range(len(users_profiles))})
profiles_numeric[0][0] = np.zeros((len(user_numeric_columns),)).astype(int)
profiles_numeric.append(None)
profiles_category.append({users_profiles.values[i, 0] : users_profiles[user_category_columns].values[i] for i in range(len(users_profiles))})
profiles_category[0][0] = np.zeros((len(user_category_columns),)).astype(int)
profiles_category.append({items_profiles.values[i, 0] : items_profiles[item_category_columns].values[i] for i in range(len(items_profiles))})
profiles_category[1][0] = np.zeros((len(item_category_columns),)).astype(int)

In [140]:
data

,user,item,rating,timestamp
1000138,12080,1697,4,956703932
1000153,12080,4633,4,956703954
999873,12080,1181,5,956703954
1000007,12080,3787,4,956703977
1000192,12080,3903,5,956703977
...,...,...,...,...
825793,9916,4663,1,1046454338
825438,9916,2771,5,1046454443
825724,9916,6393,4,1046454548
825731,9916,5133,3,1046454548


In [141]:
max_size_embedding_id = max(data["user"].max(), 0)
max_size_embedding_id = max(data["item"].max(), max_size_embedding_id)

In [142]:
max_size_embedding_id = max_size_embedding_id + 1

In [143]:
max_size_embedding_id

12081

In [144]:
max_size_embedding_category = max_size_embedding_category + 1

In [145]:
max_size_embedding_category

404

In [146]:
edges = data[data["rating"] >= 3][["user", "item"]]

In [147]:
def split_data(data, source_column, target_column, limit, split = 0.8, check = True):
    if check:
      data_groupby_source_column = data.groupby(source_column, as_index = False).agg(lambda x : [elem for elem in x])
    else:
      data_groupby_source_column = data.groupby(source_column, as_index = False).agg(lambda x : [elem for elem in x][::-1])
    data_train = None
    data_val = None
    def concatenate(data, new_data):
        if data is None:
            return new_data
        return np.concatenate((data, new_data))
    def to_df(data, columns):
        return pd.DataFrame(dict([(columns[i], data[:, i]) for i in range(len(columns))]))
    for i, value in enumerate(data_groupby_source_column[source_column].values):
        target_column_i = np.asarray(data_groupby_source_column[target_column].values[i]).reshape(-1, 1)
        if target_column_i.shape[0] < limit:
          continue
        source_column_i = np.full((target_column_i.shape[0], 1), value)
        rating_i = np.asarray(data_groupby_source_column["rating"].values[i]).reshape(-1, 1)
        if source_column == "user":
            data_i = np.hstack((source_column_i, target_column_i))
        else:
            data_i = np.hstack((target_column_i, source_column_i))
        data_i = np.hstack((data_i, rating_i))
        split_i = int(split * data_i.shape[0])
        if split_i > 0:
            data_val = concatenate(data_val, data_i[split_i:])
        else:
            split_i = data_i.shape[0]
        data_train = concatenate(data_train, data_i[:split_i])
    data_train = to_df(data_train, data.columns[:-1])
    data_val = to_df(data_val, data.columns[:-1])
    return data_train, data_val

In [148]:
limit = 0

In [149]:
#data, data_val = split_data(data, "user", "item", limit, split = split, check = True)

In [150]:
split_test = 0.8
split_train_val = 0.7

In [151]:
data_test = data[int(split_test * len(data)) : ]
data_val = data[int(split_train_val * len(data)) : int(split_test * len(data))]
data = data[:int(split_train_val * len(data))]

In [152]:
data

,user,item,rating,timestamp
1000138,12080,1697,4,956703932
1000153,12080,4633,4,956703954
999873,12080,1181,5,956703954
1000007,12080,3787,4,956703977
1000192,12080,3903,5,956703977
...,...,...,...,...
334424,3948,1819,3,974862363
334541,3948,5249,5,974862363
188106,2334,23,4,974862369
187208,2324,2459,3,974862386


In [153]:
def get_input_output(data, profiles_category, profiles_numeric):
    Y = None
    try:
        Y = data["rating"].values
    except:
        Y = None
    data_new = [data["user"].values, data["item"].values]
    for i in range(2):
        data_category_i = None
        data_numeric_i = None
        data_i = data_new[i].reshape(data_new[i].shape[0], -1, 1)
        if profiles_category[i] is not None:
            data_category_i = np.array([profiles_category[i][(val if val in profiles_category[i] else 0)] for val in data_new[i]]).reshape(data_new[i].shape[0], -1, n_category[i])
            data_i = np.dstack((data_i, data_category_i))
        if profiles_numeric[i] is not None:
            data_numeric_i = np.array([profiles_numeric[i][(val if val in profiles_numeric[i][i] else 0)] for val in data_new[i]]).reshape(data_new[i].shape[0], -1, n_numeric[i])
            data_i = np.dstack((data_i, data_numeric_i))
        data_new[i] = data_i        
    def get_full_input_2layer(data_id, i, profiles_category = None, profiles_numeric = None):
        last_input = data_id[i]
        return [torch.from_numpy(last_input[:, :, :1].astype(int)),
                torch.from_numpy(last_input[:, :, 1 : 1 + n_category[i % 2]].astype(int)),
                torch.from_numpy(last_input[:, :, 1 + n_category[i % 2]:].astype(np.float32))]
    def remove_none_or_zero_shape(list_input):
        i = 0
        while i < len(list_input):
            if list_input[i] is None:
                list_input = list_input[:i] + list_input[(i + 1):]
                continue
            if list_input[i].shape[-1] == 0:
                list_input = list_input[:i] + list_input[(i + 1):]
                continue
            i += 1
        return list_input
    input_first_type = get_full_input_2layer(data_new, 0, profiles_numeric, profiles_category)
    input_second_type = get_full_input_2layer(data_new, 1)
    if Y is not None:
        return data, input_first_type, input_second_type, torch.from_numpy(Y.reshape(-1, 1).astype(np.float32))
    return data, input_first_type, input_second_type, None

In [154]:
data, input_first_type, input_second_type, Y = get_input_output(data, profiles_category, profiles_numeric)

In [155]:
edges

,user,item
1000138,12080,1697
1000153,12080,4633
999873,12080,1181
1000007,12080,3787
1000192,12080,3903
...,...,...
825526,9916,6843
825438,9916,2771
825724,9916,6393
825731,9916,5133


In [156]:
input_second_type[0].shape

torch.Size([700146, 1, 1])

In [157]:
data_val, input_first_type_val, input_second_type_val, Y_val = get_input_output(data_val, profiles_category, profiles_numeric)

In [158]:
n_embedding = 32
n_layers = 2
lr = 1e-3
n_heads = 2
out_channels_attention = int(n_embedding / n_heads)

In [159]:
epochs = 2
batch_size = 256
validation_split = 0.1
dense_layers_nodes = [1]
class regularizers():
    def __init__(self, kernel, bias, activity):
        self.kernel = kernel
        self.bias = bias
        self.activity = activity
dense_regularizer = regularizers([1e-5, 1e-3], [1e-5, 1e-3], [1e-5, 1e-3])

In [160]:
import torch

In [161]:
device = torch.device("cuda:0")

In [162]:
edges = torch.from_numpy(edges.values.astype(int)).to(device)
nodes = torch.from_numpy(np.linspace(0, max_size_embedding_id - 1, max_size_embedding_id).astype(int)).to(device)

In [163]:
nodes

tensor([    0,     1,     2,  ..., 12078, 12079, 12080], device='cuda:0')

In [164]:
edges.shape

torch.Size([836478, 2])

In [165]:
from scipy.sparse import csr_matrix

In [166]:
class LSTMAggregation2(Aggregation):
    """Performs LSTM-style aggregation in which the elements to aggregate are
    interpreted as a sequence, as described in the `"Inductive Representation
    Learning on Large Graphs" <https://arxiv.org/abs/1706.02216>`_ paper.
    .. warning::
        :class:`LSTMAggregation` is not a permutation-invariant operator.
    Args:
        in_channels (int): Size of each input sample.
        out_channels (int): Size of each output sample.
        **kwargs (optional): Additional arguments of :class:`torch.nn.LSTM`.
    """
    def __init__(self, in_channels: int, out_channels: int, device, **kwargs):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.lstm = LSTM(in_channels, out_channels, batch_first=True, **kwargs)
        self.reset_parameters()
        self.device = device

    def reset_parameters(self):
        self.lstm.reset_parameters()

    def forward(self, x: Tensor, index: Optional[Tensor] = None,
                ptr: Optional[Tensor] = None, dim_size: Optional[int] = None,
                dim: int = -2) -> Tensor:
        #print(x)
        #print(x.shape)
        x = x.view(-1, x.shape[1] * x.shape[2])
        index_new, sorted_indices = torch.sort(index)
        print(x[sorted_indices].shape)
        x, _ = self.to_dense_batch(x[sorted_indices], index_new.to(self.device), ptr, dim_size, dim)
        return self.lstm(x)[0][:, -1]

    def __repr__(self) -> str:
        return (f'{self.__class__.__name__}({self.in_channels}, '
                f'{self.out_channels})')

In [167]:
for i in range(3952):
  if len(items_profiles[(items_profiles["item"] - 1)/2 == i]) == 0:
    print(i)

0
3884
3885
3886
3887
3888
3889
3890
3891
3892
3893
3894
3895
3896
3897
3898
3899
3900
3901
3902
3903
3904
3905
3906
3907
3908
3909
3910
3911
3912
3913
3914
3915
3916
3917
3918
3919
3920
3921
3922
3923
3924
3925
3926
3927
3928
3929
3930
3931
3932
3933
3934
3935
3936
3937
3938
3939
3940
3941
3942
3943
3944
3945
3946
3947
3948
3949
3950
3951


In [168]:
def get_test_data(data_train_new, data_new, k):
    inputs_user = data_new["user"].unique()
    new_inputs_users = np.linspace(0, inputs_user.shape[0] - 1, inputs_user.shape[0]).astype(int).reshape(-1, 1)
    to_new_id = dict(np.hstack((inputs_user.reshape(-1, 1), new_inputs_users)))
    rating = data_new["rating"].values 
    #n_item = (items_profiles["item"].max() - 1)/max_len_id
    #n_item = int(n_item)
    n_item = len(items_profiles["item"])
    true_output = csr_matrix((np.ones((rating.shape[0],)), (data_new["user"].apply(lambda x : to_new_id[x]).values,
                                                         ((data_new["item"].values - 1) / max_len_id).astype(int) - 1)),
                             shape = (inputs_user.shape[0], n_item))
    data_train_new["user"] = data_train_new["user"].apply(lambda x : (to_new_id[x] if x in to_new_id else -1))
    data_train_new = data_train_new[data_train_new["user"] != -1]
    train_output = csr_matrix((np.ones((len(data_train_new),)), (data_train_new["user"].values,
                                                                ((data_train_new["item"].values - 1) / max_len_id).astype(int) - 1)),
                               shape = (inputs_user.shape[0], n_item)).toarray()
    train_output = torch.tensor(train_output, dtype = torch.long)
    items_full = items_profiles[["item"]].copy()
    items_full["user"] = items_full["item"]
    full_test = None
    for i in range(inputs_user.shape[0]):
        items_full["user"] =  inputs_user[i]
        try:
            full_test = pd.concat([full_test, items_full.copy()])
        except:
            full_test = items_full.copy()
    data, input_first_type, input_second_type, Y = get_input_output(full_test, profiles_category, profiles_numeric)
    split_data = 1
    _, _, pred_output, loss = train_or_test(model, optimizer, input_first_type, input_second_type, Y, split_data, "test", input_first_type[0].shape[0], device)
    new_pred_output = pred_output.clone().view(inputs_user.shape[0], -1).detach().cpu()
    new_pred_output[train_output == 1.] = 0.
    recall_score = np.sum(recall_metrics(new_pred_output, true_output, k))
    ndcg_score = np.sum(ndcg_metrics(new_pred_output, true_output, k))
    return inputs_user.shape[0], recall_score, ndcg_score, loss

In [169]:
edges.shape

torch.Size([836478, 2])

In [170]:
#LSTMAggregation(n_embedding, n_embedding)
#LSTMAggregation2(n_embedding, n_embedding, device)
def get_model(): 
    class GAT_LSTM(Module):
      def __init__(self, n_embedding, n_layers, n_numeric, dense_layers_nodes, n_category, n_heads, out_channels_attention):
        super(GAT_LSTM, self).__init__()
        self.n_types = 2
        self.embedding_layer = Embedding(max_size_embedding_id, n_embedding, padding_idx = 0)
        self.embedding_layer_category = Embedding(max_size_embedding_category, n_embedding, padding_idx = 0)
        if n_numeric[0] > 0:
            self.dense_user_numeric_layer = Linear(n_numeric[0], n_embedding)
        else:
            self.dense_user_numeric_layer = None
        if n_numeric[1] > 0:
            self.dense_item_numeric_layer = Linear(n_numeric[1], n_embedding)
        else:
            self.dense_item_numeric_layer = None
        self.gnn_layer =   GraphSAGE(in_channels = n_embedding, 
                                     hidden_channels = n_embedding,
                                     num_layers = 1,
                                     aggr = MultiAggregation(['mean']))
        self.dense_layers = nn.ModuleList([Linear((2 * n_embedding if i == 0 else dense_layers_nodes[i - 1]),
                                  layer_nodes) for i, layer_nodes in enumerate(dense_layers_nodes)])
        self.weight_averaged_features_layer =  Linear(2 * n_embedding, 2)   
        self.dropout_output = nn.Dropout(0.2)
        self.n_layers = 1
        self.n_embedding = n_embedding
        self.n_numeric = n_numeric
        self.dense_layers_nodes = dense_layers_nodes
        self.check = True
        self.n_category = n_category
        self.max_size_embedding_id = max_size_embedding_id
      def get_category_features(self, inputs_category, device):
        embedding_category = self.embedding_layer_category(inputs_category)
        n_fake_category = inputs_category.shape[-1]
        n_category_inputs = 1. / torch.sum(inputs_category > 0, axis = -1)
        n_category_inputs = torch.nan_to_num(n_category_inputs)
        n_category_inputs_full = torch.full((n_fake_category,) + n_category_inputs.shape, 0, dtype = torch.float32).to(device)
        n_category_inputs_full += n_category_inputs
        n_category_inputs_full = n_category_inputs_full.permute(1, 2, 0)
        n_category_inputs_full = n_category_inputs_full.to(device)
        embedding_category = torch.matmul(embedding_category.permute(0, 1, 3, 2), 
                                          n_category_inputs_full.view(n_category_inputs_full.shape + (1,))).permute(0, 1, 3, 2)
        return embedding_category
            
      def forward(self, nodes, edges, inputs_user, inputs_user_numeric, inputs_user_category, inputs_item, inputs_item_numeric, inputs_item_category, device):
        output_user_i = None
        output_item_i = None
        def random_batch(batch_size, limit):
            limit = 0.2
            random = np.random.rand(batch_size, 1)
            random[random <= limit] = 0.
            random[random > limit] = 1.
            random = np.full((self.n_embedding,) + random.shape, random)
            random = np.transpose(random, (1, 2, 0))
            return torch.from_numpy(random.astype(np.float32))
        random_users = random_batch(inputs_user[0].shape[0], 0.3).to(device)
        random_items = random_batch(inputs_item[0].shape[0], 0.3).to(device)
        embeddings = self.embedding_layer(nodes)
        embeddings = self.gnn_layer(embeddings, edges.transpose(1, 0))
        inputs_user_new = inputs_user.detach().cpu().numpy()
        inputs_item_new = inputs_item.detach().cpu().numpy()
        inputs_user_new[inputs_user_new >= self.max_size_embedding_id] = 0
        inputs_item_new[inputs_item_new >= self.max_size_embedding_id] = 0
        embedding_users = embeddings[inputs_user_new.reshape(-1)].view(inputs_user.shape[0], 1, -1)
        embedding_items = embeddings[inputs_item_new.reshape(-1)].view(inputs_item.shape[0], 1, -1)
        #print(embedding_users[torch.isnan(embedding_users.clone().detach().cpu())])
        #print(embedding_items[torch.isnan(embedding_items.clone().detach().cpu())])
        count = 1
        user_features = None
        if self.check and ((self.n_category[0] > 0) or (self.n_numeric[0] > 0)):
            embedding_users *= random_users
            count = 0
        if self.n_category[0] > 0:
            count += 1
            embedding_users_category = self.get_category_features(inputs_user_category, device)[:, :, 0, :]
            try:
                user_features += embedding_users_category
            except:
                user_features = embedding_users_category
            #print(embedding_users_category[torch.isnan(embedding_users_category.clone().detach().cpu())])
        if self.n_numeric[0] > 0:
            embedding_users_numeric = self.dense_user_numeric_layer(inputs_user_numeric)
            embedding_users_numeric = F.relu(embedding_users_numeric)
            try:
                user_features += embedding_users_numeric
            except:
                user_features = embedding_users_numeric
            count += 1
            #print(embedding_users_numeric[torch.isnan(embedding_users_numeric.clone().detach().cpu())])
        if user_features is not None:
            user_features /= self.n_numeric[0] + self.n_category[0]
            try:
                concat_embedding_users = torch.concat((embedding_users, user_features), axis = -1)
                weight_averaged_users = self.weight_averaged_features_layer(concat_embedding_users)
                weight_averaged_users = nn.Softmax(dim = -1)(weight_averaged_users)
                concat_embedding_users = concat_embedding_users.view(concat_embedding_users.shape[0], concat_embedding_users.shape[1], 2, self.n_embedding)
                concat_embedding_users = torch.matmul(concat_embedding_users.permute(0, 1, 3, 2), weight_averaged_users.view(weight_averaged_users.shape[0],
                                                                                                                             weight_averaged_users.shape[1],
                                                                                                                             weight_averaged_users.shape[2],
                                                                                                                             1))
                embedding_users = concat_embedding_users.permute(0, 1, 3, 2)[:, :, 0, :]
            except:
                embedding_users = user_features
        if self.check and ((self.n_category[0] > 0) or (self.n_numeric[0] > 0)):
            embedding_users /= (random_users + count)
        else:
            embedding_users /= count      
        count = 1
        items_features = None
        if self.check and ((self.n_category[1] > 0) or (self.n_numeric[1] > 0)):
            embedding_items *= random_items
            count = 0
        if self.n_category[1] > 0:
            count += 1
            embedding_items_category = self.get_category_features(inputs_item_category, device)[:, :, 0, :]
            try:
                items_features += embedding_items_category
            except:
                items_features = embedding_items_category
        if self.n_numeric[1] > 0:
            embedding_items_numeric = self.dense_item_numeric_layer(inputs_item_numeric)
            embedding_items_numeric = F.relu(embedding_items_numeric)
            try:
                items_features += embedding_items_numeric
            except:
                items_features = embedding_items_numeric
            count += 1
        if items_features is not None:
            items_features /= self.n_numeric[1] + self.n_category[1]
            try:
                concat_embedding_items = torch.concat((embedding_items, items_features), axis = -1)
                weight_averaged_items = self.weight_averaged_features_layer(concat_embedding_items)
                weight_averaged_items = nn.Softmax(dim = -1)(weight_averaged_items)
                concat_embedding_items = concat_embedding_items.view(concat_embedding_items.shape[0], concat_embedding_items.shape[1], 2, self.n_embedding)
                concat_embedding_items = torch.matmul(concat_embedding_items.permute(0, 1, 3, 2), weight_averaged_items.view(weight_averaged_items.shape[0],
                                                                                                                             weight_averaged_items.shape[1],
                                                                                                                             weight_averaged_items.shape[2],
                                                                                                                             1))
                embedding_items = concat_embedding_items.permute(0, 1, 3, 2)[:, :, 0, :]
            except:
                embedding_items = items_features
        if self.check and ((self.n_category[1] > 0) or (self.n_numeric[1] > 0)):
            embedding_items /= (random_items + count)
        else:
            embedding_items /= count
        concat_output = torch.cat((embedding_users.view(-1, self.n_embedding), embedding_items.view(-1, self.n_embedding)), axis = -1)
        #print(concat_output[torch.isnan(concat_output.clone().detach().cpu())])
        #dot_output = torch.matmul()
        for i, layer in enumerate(self.dense_layers):
            #concat_output = self.dropout_output(concat_output)
            concat_output = layer(concat_output)
            concat_output = F.relu(concat_output)
        return concat_output + 1.
    model = GAT_LSTM(n_embedding, n_layers, n_numeric, dense_layers_nodes, n_category, n_heads, out_channels_attention)
    return model
model = get_model()
model = model.to(device)

In [171]:
def train_or_test(model, optimizer, input_first_type, input_second_type, Y, split_data, train_or_test, batch_size, device):
    sum_loss = 0
    random_batch = np.random.permutation(split_data)
    for j in random_batch:
        inputs_user =  input_first_type[0][j * batch_size : (j + 1) * batch_size]
        inputs_user_category =  input_first_type[1][j * batch_size : (j + 1) * batch_size].to(device)
        inputs_user_numeric = input_first_type[2][j * batch_size : (j + 1) * batch_size].to(device)
        inputs_item =  input_second_type[0][j * batch_size : (j + 1) * batch_size]
        inputs_item_category =  input_second_type[1][j * batch_size : (j + 1) * batch_size].to(device)
        inputs_item_numeric = input_second_type[2][j * batch_size : (j + 1) * batch_size].to(device)
        if Y is not None:
            if train_or_test == "train":
                pred_output = model.forward(nodes, edges, inputs_user, inputs_user_numeric, inputs_user_category, inputs_item, inputs_item_numeric, inputs_item_category, device)
                loss = nn.MSELoss()(pred_output, true_output)
                model.zero_grad()
                loss.backward()
                optimizer.step()
            else:
                with torch.no_grad():
                    pred_output = model.forward(nodes, edges, inputs_user, inputs_user_numeric, inputs_user_category, inputs_item, inputs_item_numeric, inputs_item_category, device)
                    loss = nn.MSELoss()(pred_output, true_output)
            sum_loss += loss * true_output.shape[0]
            del true_output
        else:
            with torch.no_grad():
                pred_output = model.forward(nodes, edges, inputs_user, inputs_user_numeric, inputs_user_category, inputs_item, inputs_item_numeric, inputs_item_category, device)
        del inputs_user, inputs_user_category, inputs_user_numeric, inputs_item, inputs_item_category, inputs_item_numeric
    if Y is not None:
        sum_loss = sum_loss/input_first_type[0].shape[0]
    if train_or_test == "train":print("Epoch {} train: mse_loss = {}, rmse_loss = {}".format(iter + 1, sum_loss, sum_loss**0.5))
    return model, optimizer, pred_output.clone(), sum_loss * input_first_type[0].shape[0]

In [172]:
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3, weight_decay = 1e-4)

In [173]:
#torch.autograd.set_detect_anomaly(True)

In [ ]:
#model.check = False
new_lr = lr
batch_user = 100
users_id_max = users_profiles["user"].max()
split_data = int(input_first_type[0].shape[0] / batch_size) + (0 if input_first_type[0].shape[0] % batch_size == 0 else 1)
split_data_val = int(users_id_max / batch_user) + (0 if users_id_max % batch_user == 0 else 1)
split_data_val_batch = int(input_first_type_val[0].shape[0] / batch_size) + (0 if input_first_type_val[0].shape[0] % batch_size == 0 else 1)
save_params = None
best_loss = 1e9
k = 20
epochs = 50
for iter in range(epochs):
    if iter % 5 == 0 and iter > 0:
        new_lr = new_lr * 1e-1
        optimizer = torch.optim.Adam(model.parameters(), lr = new_lr, weight_decay = 1e-4)
    sum_loss = 0
    sum_rmse = 0
    random_batch = np.random.permutation(split_data)
    for j in random_batch:
        inputs_user =  input_first_type[0][j * batch_size : (j + 1) * batch_size].to(device)
        inputs_user_category =  input_first_type[1][j * batch_size : (j + 1) * batch_size].to(device)
        inputs_user_numeric = input_first_type[2][j * batch_size : (j + 1) * batch_size].to(device)
        inputs_item =  input_second_type[0][j * batch_size : (j + 1) * batch_size].to(device)
        inputs_item_category =  input_second_type[1][j * batch_size : (j + 1) * batch_size].to(device)
        inputs_item_numeric = input_second_type[2][j * batch_size : (j + 1) * batch_size].to(device)
        true_output = Y[j * batch_size : (j + 1) * batch_size].to(device)
        pred_output = model.forward(nodes, edges, inputs_user, inputs_user_numeric, inputs_user_category, inputs_item, inputs_item_numeric, inputs_item_category, device)
        loss = MSELoss()(pred_output, true_output)
        model.zero_grad()
        loss.backward()
        optimizer.step()
        sum_loss += loss * true_output.shape[0]
    sum_loss = sum_loss/input_first_type[0].shape[0]
    print("Epoch {}: mse_train = {}, rmse_train = {}".format(iter + 1, sum_loss, sum_loss**0.5))
    model.check = False
    sum_loss = 0
    sum_rmse = 0
    random_batch_val = np.random.permutation(split_data_val_batch)
    for j in random_batch_val:
        inputs_user =  input_first_type_val[0][j * batch_size : (j + 1) * batch_size].to(device)
        inputs_user_category =  input_first_type_val[1][j * batch_size : (j + 1) * batch_size].to(device)
        inputs_user_numeric = input_first_type_val[2][j * batch_size : (j + 1) * batch_size].to(device)
        inputs_item =  input_second_type_val[0][j * batch_size : (j + 1) * batch_size].to(device)
        inputs_item_category =  input_second_type_val[1][j * batch_size : (j + 1) * batch_size].to(device)
        inputs_item_numeric = input_second_type_val[2][j * batch_size : (j + 1) * batch_size].to(device)
        true_output = Y_val[j * batch_size : (j + 1) * batch_size].to(device)
        with torch.no_grad():
          pred_output = model.forward(nodes, edges, inputs_user, inputs_user_numeric, inputs_user_category, inputs_item, inputs_item_numeric, inputs_item_category, device)
          pred_output[pred_output > 5.] = 5.
          loss = MSELoss()(pred_output, true_output)
        sum_loss += loss * true_output.shape[0]
    sum_loss = sum_loss/input_first_type_val[0].shape[0]
    print("Epoch {}: mse_val = {}, rmse_val = {}".format(iter + 1, sum_loss, sum_loss**0.5))
    random_batch_val = np.random.permutation(split_data_val)
    count = 0.
    ndcg = 0.
    recall = 0.
    sum_loss = 0.
    for j in random_batch_val:
      data_new = data_val[(data_val["user"] >= j * batch_user) & (data_val["user"] < (j + 1) * batch_user)]
      data_train_new = data[(data["user"] >= j * batch_user) & (data["user"] < (j + 1) * batch_user)]
      if len(data_new) == 0:
         continue
      n_users, new_recall, new_ndcg, loss = get_test_data(data_train_new, data_new, k)
      count += n_users
      recall += new_recall
      ndcg += new_ndcg
      sum_loss += loss
    print("Epoch {}: ndcg_val = {}, recall_val = {}, rmse_val = {}".format(iter + 1, recall / count, ndcg / count, (sum_loss / len(data_val)) ** 0.5))
    model.check = True

Epoch 1: mse_train = 1.6391843557357788, rmse_train = 1.280306339263916
Epoch 1: mse_val = 1.0070228576660156, rmse_val = 1.0035052299499512


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


Epoch 1: ndcg_val = 0.00031746031746031746, recall_val = 0.00023066761965550333, rmse_val = 0.0
Epoch 2: mse_train = 1.0159645080566406, rmse_train = 1.0079506635665894
Epoch 2: mse_val = 0.9799171090126038, rmse_val = 0.9899076223373413
Epoch 2: ndcg_val = 0.00015873015873015876, recall_val = 0.0001393839834949753, rmse_val = 0.0
Epoch 3: mse_train = 0.9598219394683838, rmse_train = 0.9797050356864929
Epoch 3: mse_val = 0.9824811816215515, rmse_val = 0.9912018775939941
Epoch 3: ndcg_val = 0.00037037037037037035, recall_val = 0.00027715159224638773, rmse_val = 0.0
Epoch 4: mse_train = 0.9415664672851562, rmse_train = 0.9703434705734253
Epoch 4: mse_val = 0.9632788300514221, rmse_val = 0.9814676642417908
Epoch 4: ndcg_val = 0.023778504904235904, recall_val = 0.016160382779032416, rmse_val = 0.0
Epoch 5: mse_train = 0.9301698207855225, rmse_train = 0.9644531011581421
Epoch 5: mse_val = 0.9648116827011108, rmse_val = 0.9822483062744141
Epoch 5: ndcg_val = 0.048065923075383016, recall_val 

KeyboardInterrupt: ignored

In [ ]:
len(items_profiles)

In [ ]:
model.parameters()

In [ ]:
model.check = False

In [ ]:
#data_test = data_test[data_test["rating"] >= 3.]

In [ ]:
#data_test["user"] = data_test["user"].apply(lambda x : x * max_len_id)
#data_test["item"] = data_test["item"].apply(lambda x : x * max_len_id + 1)

In [ ]:
data_test

In [ ]:
data_test_cold = None

In [ ]:
items_profiles["item"].values

In [ ]:
data_test

In [ ]:
data

In [ ]:
for user in data_test["user"].unique():
    if user not in data["user"].unique():
        try:
            data_test_cold = pd.concat([data_test_cold, data_test[data_test["user"] == user]])
        except:
            data_test_cold = data_test[data_test["user"] == user]
        data_test = data_test[data_test["user"] != user]

In [ ]:
data_test_cold

In [ ]:
data_test

In [ ]:
data_test, input_first_type_test, input_second_type_test, Y_test = get_input_output(data_test, profiles_category, profiles_numeric)

In [ ]:
data_test

In [ ]:
data_test_cold, input_first_type_test_cold, input_second_type_test_cold, Y_test_cold = get_input_output(data_test_cold, profiles_category, profiles_numeric)

In [ ]:
data_test_cold

In [ ]:
random_batch_test_rmse = np.random.permutation(int(input_first_type_test[0].shape[0] / batch_size) + (0 if input_first_type_test[0].shape[0] % batch_size == 0 else 1))

In [ ]:
sum_loss_test = 0
split_data_val = int(data_test["user"].max() / batch_user) + (0 if int(data_test["user"].max()) % batch_user == 0 else 1)
random_batch_test = np.random.permutation(split_data_val)
count = 0.
ndcg = 0.
recall = 0.
sum_loss = 0
for j in random_batch_test:
  data_new = data_test[(data_test["user"] >= j * batch_user) & (data_test["user"] < (j + 1) * batch_user)]
  data_train_new = data[(data["user"] >= j * batch_user) & (data["user"] < (j + 1) * batch_user)]
  if len(data_new) == 0:
    continue
  n_users, new_recall, new_ndcg, loss = get_test_data(data_train_new, data_new, 20)
  count += n_users
  recall += new_recall
  ndcg += new_ndcg
  sum_loss += loss
print("Epoch {}: ndcg_test = {}, recall_test = {}, rmse_test = {}".format(iter + 1, recall / count, ndcg / count, (sum_loss / len(data_test))**0.5))

In [ ]:
batch_user

In [ ]:
data_test_cold["user"].max()

In [ ]:
sum_loss_test_cold = 0
split_data_test_cold = int(data_test_cold["user"].max() / batch_user) + (0 if int(data_test_cold["user"].max()) % batch_user == 0 else 1)
random_batch_test_cold = np.random.permutation(split_data_test_cold)
count = 0.
ndcg = 0.
recall = 0.
sum_loss = 0
for j in random_batch_test:
  data_new = data_test_cold[(data_test_cold["user"] >= j * batch_user) & (data_test_cold["user"] < (j + 1) * batch_user)]
  data_train_new = data[(data["user"] >= j * batch_user) & (data["user"] < (j + 1) * batch_user)]
  if len(data_new) == 0:
    continue
  n_users, new_recall, new_ndcg, loss = get_test_data(data_train_new, data_new, 20)
  count += n_users
  recall += new_recall
  ndcg += new_ndcg
  sum_loss += loss
print("Epoch {}: ndcg_test = {}, recall_test = {}, rmse_test = {}".format(iter + 1, recall / count, ndcg / count, (sum_loss / len(data_test_cold))**0.5))

In [ ]:
recall

In [ ]:
sum_loss = 0
for j in random_batch_test_rmse:
    inputs_user =  input_first_type_test[0][j * batch_size : (j + 1) * batch_size].to(device)
    inputs_user_category =  input_first_type_test[1][j * batch_size : (j + 1) * batch_size].to(device)
    inputs_user_numeric = input_first_type_test[2][j * batch_size : (j + 1) * batch_size].to(device)
    inputs_item =  input_second_type_test[0][j * batch_size : (j + 1) * batch_size].to(device)
    inputs_item_category =  input_second_type_test[1][j * batch_size : (j + 1) * batch_size].to(device)
    inputs_item_numeric = input_second_type_test[2][j * batch_size : (j + 1) * batch_size].to(device)
    true_output = Y_test[j * batch_size : (j + 1) * batch_size].to(device)
    with torch.no_grad():
      pred_output = model.forward(nodes, edges, inputs_user, inputs_user_numeric, inputs_user_category, inputs_item, inputs_item_numeric, inputs_item_category, device)
      pred_output[pred_output > 5.] = 5.
      loss = MSELoss()(pred_output, true_output)
    sum_loss += loss * true_output.shape[0]
sum_loss = sum_loss/input_first_type_test[0].shape[0]
print("Epoch {}: mse_val = {}, rmse_val = {}".format(iter + 1, sum_loss, sum_loss**0.5))